In [1]:
import arcpy
import os
import pandas as pd
import numpy as np
from arcpy import env
from arcpy.management import *
from arcpy.conversion import *
from arcpy.da import *
from arcpy.sa import *

In [2]:
# 辅助函数
# 栅格转面
def raster_to_polygon(in_raster, out_polygon, field_name):
    """栅格转面"""
    in_raster = arcpy.sa.Int(in_raster)
    output_coordinate_system = arcpy.Describe(in_raster).spatialReference
    with arcpy.EnvManager(outputCoordinateSystem=output_coordinate_system):
        arcpy.conversion.RasterToPolygon(in_raster, out_polygon, "SIMPLIFY", f"{field_name}", "SINGLE_OUTER_PART", None)

# 添加字段
def add_field(input_table,field_name,field_type='TEXT'):
    """参数说明：
        input_table: 输入数据表
        field_name: 字段名
        field_type: 字段类型"""
    arcpy.AddField_management(input_table,field_name,field_type)

In [3]:
# 构建转换字典
table_path = r"F:\cache_data\frequency_filter\dy\scaler_csv\dy_frequency_filter_2024_424.csv"

In [4]:
df = pd.read_csv(table_path)

In [5]:
# 对每个唯一的土种编号生成一个字典
def aggregate_rows(group):
    records = group[['TL', 'YL', 'TS']].to_dict('records')
    # 假设每个土种编号下的记录是相同的，这里我们取第一个记录
    return records[0] if records else {}

# 使用groupby按照`土种编号`聚合，并应用自定义函数
soil_dict = df.groupby('TZ').apply(aggregate_rows).to_dict()

In [6]:
tz_dict = {0: ['中层壤质中性紫色土'], 1: ['中层灰泥质黄色石灰土'], 2: ['中层砾壤质黑色石灰土'], 3: ['中层红泥质黄壤'], 4: ['厚层壤质黄色石灰土'], 5: ['厚层泥质黄壤'], 6: ['厚层红泥质黄壤'], 7: ['浅石灰泥田'], 8: ['潮泥田'], 9: ['潮砂泥田'], 10: ['石灰泥田'], 11: ['腐中层壤质酸性紫色土'], 12: ['腐中层壤质钙质紫色土'], 13: ['腐中层壤质黄色石灰土'], 14: ['腐中层泥质黄壤'], 15: ['腐中层灰泥质黄壤'], 16: ['腐中层砂泥质黄壤'], 17: ['腐中层砂泥质黄壤性土'], 18: ['腐中层硅质黄壤'], 19: ['腐厚层壤质黄色石灰土'], 20: ['腐厚层泥质黄壤'], 21: ['腐厚层灰泥质黄壤'], 22: ['腐厚层砂泥质黄壤'], 23: ['腐厚层硅质黄壤'], 24: ['腐薄层壤质黄色石灰土'], 25: ['腐薄层壤质黑色石灰土'], 26: ['腐薄层暗泥质黄壤'], 27: ['腐薄层泥质黄壤'], 28: ['腐薄层灰泥质黄壤'], 29: ['腐薄层砂泥质黄壤'], 30: ['腐薄层砾硅质黄壤'], 31: ['腐薄层硅质山地灌丛草甸土'], 32: ['腐薄层硅质黄壤'], 33: ['腐薄层黏质黄色石灰土'], 34: ['薄层壤质中性紫色土'], 35: ['薄层灰泥质黄色石灰土'], 36: ['薄层砾壤质中性紫色土'], 37: ['薄层砾壤质黑色石灰土'], 38: ['薄层砾硅质黄壤'], 39: ['薄层硅质黄壤'], 40: ['薄层红泥质黄壤'], 41: ['轻漂砂泥田'], 42: ['重漂砂泥田'], 43: ['重漂红泥田'], 44: ['青石灰泥田'], 45: ['黄浅白粉泥田'], 46: ['黄浅砂泥田'], 47: ['黄浅红泥田'], 48: ['黄浅鳝泥田'], 49: ['黄白粉泥田'], 50: ['黄砂泥田'], 51: ['黄红泥田'], 52: ['黄青白粉泥田'], 53: ['黄青砂泥田'], 54: ['黄黄砂泥田']}

In [7]:
result_dict = {}
for one_key in tz_dict:
    result_dict[one_key] = {'TZ':tz_dict[one_key][0],'TL':soil_dict[tz_dict[one_key][0]]['TL'],'TS':soil_dict[tz_dict[one_key][0]]['TS'],'YL':soil_dict[tz_dict[one_key][0]]['YL']}

In [8]:
result_dict

{0: {'TZ': '中层壤质中性紫色土', 'TL': '紫色土', 'TS': '壤质中性紫色土', 'YL': '中性紫色土'},
 1: {'TZ': '中层灰泥质黄色石灰土', 'TL': '石灰土', 'TS': '灰泥质黄色石灰土', 'YL': '黄色石灰土'},
 2: {'TZ': '中层砾壤质黑色石灰土', 'TL': '石灰土', 'TS': '砾壤质黑色石灰土', 'YL': '黑色石灰土'},
 3: {'TZ': '中层红泥质黄壤', 'TL': '黄壤', 'TS': '红泥质黄壤', 'YL': '典型黄壤'},
 4: {'TZ': '厚层壤质黄色石灰土', 'TL': '石灰土', 'TS': '壤质黄色石灰土', 'YL': '黄色石灰土'},
 5: {'TZ': '厚层泥质黄壤', 'TL': '黄壤', 'TS': '泥质黄壤', 'YL': '典型黄壤'},
 6: {'TZ': '厚层红泥质黄壤', 'TL': '黄壤', 'TS': '红泥质黄壤', 'YL': '典型黄壤'},
 7: {'TZ': '浅石灰泥田', 'TL': '水稻土', 'TS': '浅灰泥田', 'YL': '淹育型水稻土'},
 8: {'TZ': '潮泥田', 'TL': '水稻土', 'TS': '潮泥田', 'YL': '潴育型水稻土'},
 9: {'TZ': '潮砂泥田', 'TL': '水稻土', 'TS': '潮砂泥田', 'YL': '潴育型水稻土'},
 10: {'TZ': '石灰泥田', 'TL': '水稻土', 'TS': '石灰泥田', 'YL': '潴育型水稻土'},
 11: {'TZ': '腐中层壤质酸性紫色土', 'TL': '紫色土', 'TS': '壤质酸性紫色土', 'YL': '酸性紫色土'},
 12: {'TZ': '腐中层壤质钙质紫色土', 'TL': '紫色土', 'TS': '壤质钙质紫色土', 'YL': '钙质紫色土'},
 13: {'TZ': '腐中层壤质黄色石灰土', 'TL': '石灰土', 'TS': '壤质黄色石灰土', 'YL': '黄色石灰土'},
 14: {'TZ': '腐中层泥质黄壤', 'TL': '黄壤', 'TS': '泥质黄壤', 'YL': '典型

In [9]:
raster_path = r'D:\ArcGISProjects\workspace\shbyq\feature_soiltype_result\feature_soiltype_result_dy.gdb\prediction_soiltype_RFE_20240429_simple_nolonlat_smooth'
out_polygon_path = r'D:\ArcGISProjects\workspace\shbyq\soil_type_result.gdb\soiltype_dy\prediction_soiltype_RFE_20240429_simple_nolonlat_smooth'
field_name = 'Value'

In [10]:
# 栅格转面
raster_to_polygon(raster_path, out_polygon_path, field_name)

In [11]:
code_block =f"""
def get_soil_class(soil_code):
    soil_dict = {result_dict}
    TZ = soil_dict[soil_code]['TZ']
    TL = soil_dict[soil_code]['TL']
    YL = soil_dict[soil_code]['YL']
    TS = soil_dict[soil_code]['TS']
    result_list = [TZ,TS,YL,TL]
    return result_list[0]
"""
field_name = 'TZ'

In [12]:
expression = "get_soil_class(!gridcode!)"

In [13]:
# 增加字段
for one_field in ['TL','YL','TS','TZ']:
    add_field(out_polygon_path, one_field, 'TEXT')

In [14]:
# 计算字段
arcpy.CalculateField_management(in_table=out_polygon_path, field=field_name, expression=expression, expression_type="PYTHON3", code_block=code_block)
    

<Result 'D:\\ArcGISProjects\\workspace\\shbyq\\soil_type_result.gdb\\soiltype_dy\\prediction_soiltype_RFE_20240429_simple_nolonlat_smooth'>

In [15]:
code_block =f"""
def get_soil_class(soil_code):
    soil_dict = {result_dict}
    TZ = soil_dict[soil_code]['TZ']
    TL = soil_dict[soil_code]['TL']
    YL = soil_dict[soil_code]['YL']
    TS = soil_dict[soil_code]['TS']
    result_list = [TZ,TS,YL,TL]
    return result_list[1]
"""
field_name = 'TS'
# 计算字段
arcpy.CalculateField_management(in_table=out_polygon_path, field=field_name, expression=expression, expression_type="PYTHON3", code_block=code_block)

<Result 'D:\\ArcGISProjects\\workspace\\shbyq\\soil_type_result.gdb\\soiltype_dy\\prediction_soiltype_RFE_20240429_simple_nolonlat_smooth'>

In [16]:
code_block =f"""
def get_soil_class(soil_code):
    soil_dict = {result_dict}
    TZ = soil_dict[soil_code]['TZ']
    TL = soil_dict[soil_code]['TL']
    YL = soil_dict[soil_code]['YL']
    TS = soil_dict[soil_code]['TS']
    result_list = [TZ,TS,YL,TL]
    return result_list[2]
"""
field_name = 'YL'
# 计算字段
arcpy.CalculateField_management(in_table=out_polygon_path, field=field_name, expression=expression, expression_type="PYTHON3", code_block=code_block)

<Result 'D:\\ArcGISProjects\\workspace\\shbyq\\soil_type_result.gdb\\soiltype_dy\\prediction_soiltype_RFE_20240429_simple_nolonlat_smooth'>

In [17]:
code_block =f"""
def get_soil_class(soil_code):
    soil_dict = {result_dict}
    TZ = soil_dict[soil_code]['TZ']
    TL = soil_dict[soil_code]['TL']
    YL = soil_dict[soil_code]['YL']
    TS = soil_dict[soil_code]['TS']
    result_list = [TZ,TS,YL,TL]
    return result_list[3]
"""
field_name = 'TL'
# 计算字段
arcpy.CalculateField_management(in_table=out_polygon_path, field=field_name, expression=expression, expression_type="PYTHON3", code_block=code_block)

<Result 'D:\\ArcGISProjects\\workspace\\shbyq\\soil_type_result.gdb\\soiltype_dy\\prediction_soiltype_RFE_20240429_simple_nolonlat_smooth'>